**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [57]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
from keras.layers import Flatten


from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [58]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [59]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns the next action to do.
It computes the $\epsilon$-greedy policy:
the algorithm chooses a random action with propability $\epsilon$ 
and returns the action that maximizes the $Q$ function in state $s$
with propability $1-\epsilon$.

$\epsilon$ is essential to allow exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [60]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [64]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is the array such as:
```position[x,y]=1``` if the agent is in the position $(x,y)$,
```position[x,y]=0``` if the agent is not in the position $(x,y)$

```board``` is the array which represents the rewards for each state:
```board[x,y]``` is the reward for the state $s=(x,y)$

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [65]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [94]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):        
        state = env.reset()
        win = 0
        lose = 0
        game_over = False
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))


In [67]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/9.0. Average score (2.0)
Win/lose count 7.0/14.0. Average score (-2.5)
Win/lose count 13.0/12.0. Average score (-1.3333333333333333)
Win/lose count 10.5/24.0. Average score (-4.375)
Win/lose count 7.5/14.0. Average score (-4.8)
Win/lose count 6.0/10.0. Average score (-4.666666666666667)
Win/lose count 10.0/12.0. Average score (-4.285714285714286)
Win/lose count 5.0/10.0. Average score (-4.375)
Win/lose count 10.0/16.0. Average score (-4.555555555555555)
Win/lose count 10.0/18.0. Average score (-4.9)
Win/lose count 12.0/12.0. Average score (-4.454545454545454)
Win/lose count 7.0/10.0. Average score (-4.333333333333333)
Win/lose count 11.5/18.0. Average score (-4.5)
Win/lose count 13.0/16.0. Average score (-4.392857142857143)
Win/lose count 11.0/14.0. Average score (-4.3)
Win/lose count 7.0/13.0. Average score (-4.40625)
Win/lose count 14.0/12.0. Average score (-4.029411764705882)
Win/lose count 6.5/6.0. Average score (-3.7777777777777777)
Win/lose count 12.5/11.0. Av

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Result 1__
By definition of the $Q$ function, we have:
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

$$Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
=E_{p^{\pi}}[r(s,a) + \gamma \sum_{s'} P(s_{1}=s'|s_{0}=s,a_{0}=a) \sum_{t=1}^{T}\gamma^{t-1}r(s'_{t},a'_{t})|s_{1}=s',a_{1}=\pi(s')] $$ (Markov property)
$$=E_{p^{\pi}}[r(s,a) + \gamma \sum_{s'} P(s_{1}=s'|s_{0}=s,a_{0}=a) \sum_{t=0}^{T}\gamma^{t}r(s'_{t},a'_{t})|s_{1}=s',a_{1}=\pi(s')] $$ (Change of time)
$$=E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^\pi(s',a')]$$ 

__Result 2__
$$Q^{*}(s,a)=\underset{\pi}{max} \,\, Q^{\pi}(s,a) \\
=\underset{\pi}{max} \,\, E_{p^{\pi}}[r(s,a) + \sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
=E_{p^{\pi}}[r(s,a) + \gamma \sum_{s'} P(s_{1}=s'|s_{0}=s,a_{0}=a) \,\, \underset{a'=\pi(s')}{max} \,\, \sum_{t=1}^{T}\gamma^{t-1}r(s'_{t},a'_{t})|s_{1}=s',a_{1}=\pi(s')]\\
=E_{p^{\pi}}[r(s,a) + \gamma \sum_{s'} P(s_{1}=s'|s_{0}=s,a_{0}=a) \,\, \underset{a'=\pi(s')}{max} \,\, \sum_{t=0}^{T}\gamma^{t}r(s'_{t},a'_{t})|s_{1}=s',a_{1}=\pi(s')] \\
=E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma \,\, \underset{a'=\pi(s')}{max} \,\, Q^{\pi^{*}}(s',a')]$$

__Result 3__

We want to converge to the best policy, so we minimize the expected value of the distance between the optimal $Q$ function and the $Q$ function:
To do this, we use the TD estimator of the action value function $Q$:  $r+\gamma \max_{a'}Q(s',a')$
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [68]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [69]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [70]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))
        '''p = random.random()
        eps = 0.1
        if p < eps:
            return np.random.randint(0, self.n_action, size=1)[0]
        else:
            return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))'''
            

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] = self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            input_states[i] = s_
            
            if game_over_:
                ######## FILL IN
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                q_plus = self.model.predict(np.expand_dims(n_s_, axis=0))
                target_q[i, a_] = r_ + self.discount * np.max(q_plus)
                
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), 'mse')
        self.model = model

In [71]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

In [29]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent,env,epochs_train,prefix='fc_train')

#HTML(display_videos('fc_train.mp4'))

Epoch 000/100 | Loss 0.0240 | Win/lose count 4.5/2.0 (2.5)
Epoch 001/100 | Loss 0.0164 | Win/lose count 5.0/13.0 (-8.0)
Epoch 002/100 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 003/100 | Loss 0.0192 | Win/lose count 4.5/3.0 (1.5)
Epoch 004/100 | Loss 0.0670 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/100 | Loss 0.0117 | Win/lose count 1.5/8.0 (-6.5)
Epoch 006/100 | Loss 0.0019 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/100 | Loss 0.0212 | Win/lose count 4.0/8.0 (-4.0)
Epoch 008/100 | Loss 0.0042 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/100 | Loss 0.0037 | Win/lose count 5.5/3.0 (2.5)
Epoch 010/100 | Loss 0.0028 | Win/lose count 2.5/4.0 (-1.5)
Epoch 011/100 | Loss 0.0072 | Win/lose count 3.0/6.0 (-3.0)
Epoch 012/100 | Loss 0.0035 | Win/lose count 6.5/1.0 (5.5)
Epoch 013/100 | Loss 0.0098 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/100 | Loss 0.0014 | Win/lose count 2.5/3.0 (-0.5)
Epoch 015/100 | Loss 0.0013 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/100 | Loss 0.0068 | Win/lose count 7.5

In [30]:
HTML(display_videos('fc_train10.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [72]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), 'mse')
        #model.compile(adam(lr=lr, decay=1e-4), 'mse')
        self.model = model

In [32]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')

#HTML(display_videos('fc_train.mp4'))

Epoch 000/100 | Loss 0.0005 | Win/lose count 4.0/8.0 (-4.0)
Epoch 001/100 | Loss 0.0042 | Win/lose count 2.5/2.0 (0.5)
Epoch 002/100 | Loss 0.0064 | Win/lose count 3.5/5.0 (-1.5)
Epoch 003/100 | Loss 0.0015 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/100 | Loss 0.0053 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/100 | Loss 0.0014 | Win/lose count 7.0/5.0 (2.0)
Epoch 006/100 | Loss 0.0029 | Win/lose count 11.0/8.0 (3.0)
Epoch 007/100 | Loss 0.0045 | Win/lose count 2.0/0 (2.0)
Epoch 008/100 | Loss 0.0034 | Win/lose count 8.5/3.0 (5.5)
Epoch 009/100 | Loss 0.0023 | Win/lose count 13.0/4.0 (9.0)
Epoch 010/100 | Loss 0.0012 | Win/lose count 10.0/5.0 (5.0)
Epoch 011/100 | Loss 0.0038 | Win/lose count 8.5/3.0 (5.5)
Epoch 012/100 | Loss 0.0031 | Win/lose count 12.5/0 (12.5)
Epoch 013/100 | Loss 0.0015 | Win/lose count 9.0/3.0 (6.0)
Epoch 014/100 | Loss 0.0033 | Win/lose count 12.5/5.0 (7.5)
Epoch 015/100 | Loss 0.0014 | Win/lose count 7.5/4.0 (3.5)
Epoch 016/100 | Loss 0.0011 | Win/lose count 7.0/3.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [33]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.0/0. Average score (7.0)
Win/lose count 12.5/5.0. Average score (7.25)
Win/lose count 2.5/4.0. Average score (4.333333333333333)
Win/lose count 7.5/3.0. Average score (4.375)
Win/lose count 5.5/1.0. Average score (4.4)
Win/lose count 6.5/1.0. Average score (4.583333333333333)
Win/lose count 8.5/3.0. Average score (4.714285714285714)
Win/lose count 10.5/2.0. Average score (5.1875)
Win/lose count 9.0/4.0. Average score (5.166666666666667)
Win/lose count 3.0/3.0. Average score (4.65)
Win/lose count 8.0/2.0. Average score (4.7727272727272725)
Win/lose count 10.5/4.0. Average score (4.916666666666667)
Win/lose count 4.5/1.0. Average score (4.8076923076923075)
Win/lose count 9.0/3.0. Average score (4.892857142857143)
Win/lose count 12.0/1.0. Average score (5.3)
Win/lose count 14.5/3.0. Average score (5.6875)
Win/lose count 5.5/5.0. Average score (5.382352941176471)
Win/lose count 7.5/1.0. Average score (5.444444444444445)
Win/lose count 9.5/0. Average score (

In [34]:
HTML(display_videos('cnn_test10.mp4'))

In [35]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [79]:
def train_explore(agent,env,epoch,epsilon=0.1,decay=0.99,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.epsilon *= decay
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)


        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
#class EnvironmentExploring(object):
#    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
           
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [80]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=T, temperature=0.3): #max_time=500
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #Malus
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        
        reward = reward + self.board[self.x, self.y] 
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [81]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0098 | Win/lose count 11.0/49.400000000000034 (-38.400000000000034)
Epoch 001/100 | Loss 0.0185 | Win/lose count 13.0/35.400000000000006 (-22.400000000000006)
Epoch 002/100 | Loss 0.0125 | Win/lose count 24.5/18.59999999999999 (5.900000000000009)
Epoch 003/100 | Loss 0.0241 | Win/lose count 16.5/35.200000000000024 (-18.700000000000024)
Epoch 004/100 | Loss 0.0155 | Win/lose count 24.5/15.799999999999983 (8.700000000000017)
Epoch 005/100 | Loss 0.0184 | Win/lose count 18.0/22.299999999999997 (-4.299999999999997)
Epoch 006/100 | Loss 0.0100 | Win/lose count 17.0/15.999999999999988 (1.0000000000000124)
Epoch 007/100 | Loss 0.0130 | Win/lose count 24.0/12.799999999999978 (11.200000000000022)
Epoch 008/100 | Loss 0.0409 | Win/lose count 20.5/20.50000000000001 (-1.0658141036401503e-14)
Epoch 009/100 | Loss 0.0091 | Win/lose count 27.5/20.0 (7.5)
Epoch 010/100 | Loss 0.0373 | Win/lose count 19.0/20.699999999999992 (-1.6999999999999922)
Epoch 011/100 | Loss 0.0053 | Win/

Epoch 094/100 | Loss 0.0075 | Win/lose count 18.5/16.69999999999999 (1.8000000000000114)
Epoch 095/100 | Loss 0.0026 | Win/lose count 23.5/19.499999999999996 (4.0000000000000036)
Epoch 096/100 | Loss 0.0050 | Win/lose count 24.5/23.700000000000017 (0.799999999999983)
Epoch 097/100 | Loss 0.0050 | Win/lose count 20.0/25.999999999999996 (-5.9999999999999964)
Epoch 098/100 | Loss 0.0068 | Win/lose count 19.5/22.500000000000018 (-3.0000000000000178)
Epoch 099/100 | Loss 0.0060 | Win/lose count 25.0/16.099999999999984 (8.900000000000016)


In [82]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.0/0. Average score (18.0)
Win/lose count 19.5/0. Average score (18.75)
Win/lose count 26.5/2.0. Average score (20.666666666666668)
Win/lose count 24.0/1.0. Average score (21.25)
Win/lose count 15.0/4.0. Average score (19.2)
Win/lose count 29.0/1.0. Average score (20.666666666666668)
Win/lose count 21.0/4.0. Average score (20.142857142857142)
Win/lose count 20.0/0. Average score (20.125)
Win/lose count 27.5/5.0. Average score (20.38888888888889)
Win/lose count 17.0/2.0. Average score (19.85)
Win/lose count 20.0/5.0. Average score (19.40909090909091)
Win/lose count 21.5/2.0. Average score (19.416666666666668)
Win/lose count 17.5/1.0. Average score (19.192307692307693)
Win/lose count 26.5/4.0. Average score (19.428571428571427)
Win/lose count 19.0/5.0. Average score (19.066666666666666)
Win/lose count 24.5/1.0. Average score (19.34375)
Win/lose count 24.0/2.0. Average score (19.5)
Win/lose count 21.5/0. Average score (19.61111111111111)
Win/lose count 19.0/2.0. Average s

With a progessive decrease of the epsilon during the time of the party, we diminuate the exploration which improve the final cumulative reward. Indeed, if we always explorate, we do not select the optimal action. Our technique is like the "Doubling Trick": at each time step, we multiply epsilon by a decay.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [90]:
class Mimic_DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic_DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))
        '''p = random.random()
        eps = 0.1
        if p < eps:
            return np.random.randint(0, self.n_action, size=1)[0]
        else:
            return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))'''
            

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            s_, a_  = self.memory.random_access()
            target_q[i, a_] = 5
            input_states[i] = s_

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class Mimic_DQN_CNN(Mimic_DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimic_DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), 'mse')
        #model.compile(adam(lr=lr, decay=1e-4), 'mse')
        self.model = model

In [91]:
def train_mimic_agent(expert,agent,env,epoch=100,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = expert.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [92]:
expert = agent
mimic_agent = Mimic_DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16, n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimic_agent(expert,mimic_agent, env, prefix='mimic_dqn_cnn')

Epoch 000/100 | Loss 2.3065 | Win/lose count 18.5/25.000000000000007 (-6.500000000000007)
Epoch 001/100 | Loss 2.1903 | Win/lose count 19.0/37.90000000000003 (-18.900000000000027)
Epoch 002/100 | Loss 2.0518 | Win/lose count 21.5/16.69999999999999 (4.800000000000011)
Epoch 003/100 | Loss 2.9503 | Win/lose count 27.0/19.800000000000004 (7.199999999999996)
Epoch 004/100 | Loss 2.2856 | Win/lose count 18.0/34.00000000000003 (-16.00000000000003)
Epoch 005/100 | Loss 2.3984 | Win/lose count 21.0/15.899999999999983 (5.100000000000017)
Epoch 006/100 | Loss 1.8800 | Win/lose count 22.0/21.59999999999999 (0.40000000000000924)
Epoch 007/100 | Loss 2.5415 | Win/lose count 19.0/25.6 (-6.600000000000001)
Epoch 008/100 | Loss 2.9724 | Win/lose count 24.5/15.699999999999985 (8.800000000000015)
Epoch 009/100 | Loss 2.7592 | Win/lose count 24.5/17.99999999999999 (6.500000000000011)
Epoch 010/100 | Loss 1.4850 | Win/lose count 19.0/17.299999999999983 (1.700000000000017)
Epoch 011/100 | Loss 1.9924 | Win

Epoch 093/100 | Loss 1.6263 | Win/lose count 14.5/27.300000000000004 (-12.800000000000004)
Epoch 094/100 | Loss 0.6663 | Win/lose count 21.5/16.699999999999985 (4.800000000000015)
Epoch 095/100 | Loss 1.2606 | Win/lose count 21.0/27.099999999999984 (-6.099999999999984)
Epoch 096/100 | Loss 0.2246 | Win/lose count 28.0/15.99999999999998 (12.00000000000002)
Epoch 097/100 | Loss 0.8605 | Win/lose count 21.0/20.39999999999999 (0.6000000000000085)
Epoch 098/100 | Loss 0.6899 | Win/lose count 22.5/18.199999999999996 (4.300000000000004)
Epoch 099/100 | Loss 1.1964 | Win/lose count 21.5/20.800000000000008 (0.6999999999999922)


***

In [95]:
# Evaluation of the Mimic Agent
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
print('Test of the Mimic Agent')
test(mimic_agent,env_test,epochs_test,prefix='cnn_mimic_agent_test')

Test of the Mimic Agent
Win/lose count 11.5/6.0. Average score (5.5)
Win/lose count 19.0/9.0. Average score (7.75)
Win/lose count 9.0/3.0. Average score (7.166666666666667)
Win/lose count 17.0/5.0. Average score (8.375)
Win/lose count 12.0/3.0. Average score (8.5)
Win/lose count 16.0/2.0. Average score (9.416666666666666)
Win/lose count 12.0/6.0. Average score (8.928571428571429)
Win/lose count 16.0/7.0. Average score (8.9375)
Win/lose count 19.0/10.0. Average score (8.944444444444445)
Win/lose count 12.0/9.0. Average score (8.35)
Win/lose count 14.5/4.0. Average score (8.545454545454545)
Win/lose count 10.5/2.0. Average score (8.541666666666666)
Win/lose count 21.0/5.0. Average score (9.115384615384615)
Win/lose count 8.0/4.0. Average score (8.75)
Win/lose count 15.5/7.0. Average score (8.733333333333333)
Win/lose count 3.5/0. Average score (8.40625)
Win/lose count 10.5/5.0. Average score (8.235294117647058)
Win/lose count 6.0/5.0. Average score (7.833333333333333)
Win/lose count 13.5

In [96]:
HTML(display_videos('cnn_mimic_agent_test10.mp4'))

The Mimic Agent gives a lower win/lose ratio but is faster than the Expert Agent